In [5]:
import datetime
import pandas as pd
import micromegas
pd.set_option('display.max_rows', 500)
client = micromegas.connect()

In [ ]:
%%time
sql = """
SELECT *
FROM processes
WHERE exe ilike '%flight%'
order by start_time DESC
limit 2
"""
client.query(sql)

In [8]:
sql = """
      WITH begin_events AS (
            SELECT span_id, time as begin_time, name, filename, target, line
            FROM view_instance('async_events', '{process_id}')
            WHERE event_type = 'begin'
        ),
        end_events AS (
            SELECT span_id, time as end_time
            FROM view_instance('async_events', '{process_id}')
            WHERE event_type = 'end'
        )
        SELECT 
            b.span_id,
            b.begin_time,
            e.end_time,
            b.name,
            b.filename,
            b.target,
            b.line
        FROM begin_events b
        INNER JOIN end_events e ON b.span_id = e.span_id
        ORDER BY b.begin_time
  ;""".format( process_id = "d35a4b40-7407-40cd-a1cf-c619124ee529" )
df = client.query(sql)

In [10]:
df["duration"] = df["end_time"] - df["begin_time"]

In [15]:
df["duration"].min()

Timedelta('-1 days +23:59:59.996101249')